<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat with CompressibleGroupChatManager (Experimental)

AutoGen offers conversable agents powered by LLM, tools, or humans, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use an experimental version of AutoGen's GroupChatManager, `CompressibleGroupChatManager`. If compression is turned on, the manager will check the token limit before selecting the next agent to speak and update the chat history if the limit is reached. The updated messages will be broadcast to all agents in the conversation. To use the compression feature, simply replace `GroupChatManager` with `CompressibleGroupChatManager`, and set the `compress_config`. There are different modes to be used: `TERMINATE, COMPRESS, CUSTOM`. The compressible manager inherits from `CompressibleAgent`, and you can read more about compression and compressible agents in `agentchat_compression.ipynb`.

Note: The trigger amount is compared with token count of the groupchatmanager, which will have a system message aggregated from all agents in the groupchat. This doesn't matter what groupchat selection strategy is used (manual, roundrobin, etc). This will be improved in the future.
You can print `groupchatmanager.init_count` to get the initial token count.


## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [105]:
%%capture --no-stderr
# %pip install pyautogen~=0.1.0

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Example group chat with compression

This example is from [agentchat_groupchat.ipynb](https://github.com/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb). 

Simply replace `GroupChatManager` with `CompressibleGroupChatManager` to enable compression. Refer to `agentchat_compression.ipynb` for other ways of handling long conversations.

In [2]:
import autogen
llm_config = {"config_list": config_list_gpt4, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=20)

# replace GroupChatManager with CompressibleGroupChatManager
# manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)
from autogen.agentchat.contrib.compressible_groupchatmanager import CompressibleGroupChatManager
manager = CompressibleGroupChatManager(
    groupchat=groupchat, 
    llm_config=llm_config,
    compress_config={
        "mode": "COMPRESS",
        "trigger_count": 2000,
        "leave_last_n": 2, # leave last n messages uncompressed
        "verbose": False, # to allow printing of compression information
    })

print(f"Initial token count from manager: {manager.init_token_count}")

Initial token count from manager: 567


In [3]:
user_proxy.initiate_chat(manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software.")
# type exit to terminate the chat

User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Coder (to chat_manager):

To achieve the task, we need to perform two steps:

1. We need to search the latest papers about GPT-4 on arXiv.
2. Once we have the paper details, we need to extract information related to GPT-4's potential applications in software from it.

I can help to get automated search results from arXiv using its API. However, fetching data from downloaded papers is more complex, as they are normally in PDF format. An AI model can't read and comprehend text from a PDF in the same way a human would, but I can assist you to get the paper, you would need to manually read it for detailed information.

Let's start with the first step, which is to find the latest research paper about GPT-4 on arXiv. We will use the `arxiv` python library for this, which is a Python wrapper f

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 12, in <module>
    for result in search.get():
AttributeError: 'Search' object has no attribute 'get'


--------------------------------------------------------------------------------
Coder (to chat_manager):

I apologize for the confusion. The `Search` object indeed doesn't have a `get()` method. We should iterate over the `Search` object directly.

Directly iterating over the `Search` object will give the desired result. This is because the `Search` object is an iterable and it uses `get()` method internally.

Here is the corrected Python code:

```python
# import required library
import arxiv

# search for the latest papers on GPT-4
search = arxiv.Search(
  query = "GPT-4",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

# print the latest paper details
for result in search.results():
  print(f"Title: {result.title}")
  print(f"Author(s

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
:12: DeprecationWarning: The '(Search).results' method is deprecated, use 'Client.results' instead
  for result in search.results():
Traceback (most recent call last):
  File "", line 14, in <module>
    print(f"Author(s): {', '.join(result.authors)}")
TypeError: sequence item 0: expected str instance, Author found


--------------------------------------------------------------------------------
Coder (to chat_manager):

I apologize for the confusion again. It seems there has been some changes in the way authors are represented in the latest versions of the arxiv library. The authors are now returned as `Author` objects.

Let's modify the code to handle this:

```python
# import required library
import arxiv

# search for the latest papers on GPT-4
search = arxiv.Search(
  query = "GPT-4",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

# print the latest paper details
for result in search.

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 12, in <module>
    for result in search.get():
AttributeError: 'Search' object has no attribute 'get'


--------------------------------------------------------------------------------
Coder (to chat_manager):

I apologize for the confusion earlier. The 'get()' method was inadvertently left in the code. In fact, we need to iterate over the `Search` object directly. This required change has been reflected in the code provided below.

```python
# import required library
import arxiv

# search for the latest papers on GPT-4
search = arxiv.Search(
  query = "GPT-4",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

# print the latest paper details
for result in search.results():
  print(f"Title: {result.title}")

  # Build author list as string
  authors = ", ".join([str(author) for author in result.authors])
  print(f"Author(s): {authors}")

  p

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
Title: Large Language Models as Automated Aligners for benchmarking Vision-Language Models
Author(s): Yuanfeng Ji, Chongjian Ge, Weikai Kong, Enze Xie, Zhengying Liu, Zhengguo Li, Ping Luo
Abstract: With the advancements in Large Language Models (LLMs), Vision-Language Models
(VLMs) have reached a new level of sophistication, showing notable competence
in executing intricate cognition and reasoning tasks. However, existing
evaluation benchmarks, primarily relying on rigid, hand-crafted datasets to
measure task-specific performance, face significant limitations in assessing
the alignment of these increasingly anthropomorphic models with human
intelligence. In this work, we address the limitations via Auto-Bench, which
delves into exploring LLMs as proficient aligners, measuring the alignment
between VLMs and human intelligence and value through automatic data curation
and assessment. Specifically, for data cu